# Device

In [1]:
import torch

# device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
device = torch.device('cpu')
device

device(type='cpu')

# Init model

In [2]:
from backbones import get_model

model = get_model('repvgg')
model = model.eval()

In [3]:
from torchinfo import summary
summary(model, input_size=(1, 3, 112, 112), col_names=('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'))

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
RepVGG                                   [1, 3, 112, 112]          [1, 512]                  --                        --                        --
├─RepVGGBlock: 1-1                       [1, 3, 112, 112]          [1, 48, 56, 56]           --                        --                        --
│    └─Sequential: 2-1                   [1, 3, 112, 112]          [1, 48, 56, 56]           --                        --                        --
│    │    └─Conv2d: 3-1                  [1, 3, 112, 112]          [1, 48, 56, 56]           1,296                     [3, 3]                    4,064,256
│    │    └─BatchNorm2d: 3-2             [1, 48, 56, 56]           [1, 48, 56, 56]           96                        --                        96
│    └─Sequential: 2-2                   [1, 3, 112, 112]          [1, 48, 56, 56]           --   

In [4]:
model = model.to(device)

# Generate random input

In [5]:
import numpy as np
img = np.random.rand(112, 112, 3)
img = np.transpose(img, (2, 0, 1))
img = torch.from_numpy(img).unsqueeze(0).float()
img = img.to(device)
img.shape

torch.Size([1, 3, 112, 112])

In [6]:
with torch.no_grad():
    feat = model(img)
feat

tensor([[ 3.2363e-01,  4.7822e-02,  3.1942e-02,  3.1439e-01,  2.9077e-01,
         -1.6843e-01,  2.1655e-02, -1.9590e-01,  2.0792e-02,  3.2545e-01,
          4.0848e-01,  1.9912e-01,  1.6813e-01,  2.2000e-01,  1.5470e-01,
          4.3031e-01,  4.2403e-02, -1.8386e-01,  4.2585e-01,  4.6259e-01,
         -3.7674e-01,  2.8828e-03, -8.3697e-02,  8.6850e-02, -5.1918e-02,
          1.5622e-02,  1.2021e-01,  2.3824e-01, -2.7435e-01, -3.3648e-01,
         -6.2325e-02,  9.0248e-02, -1.9050e-01,  1.3745e-01, -1.2862e-02,
          5.9203e-01, -3.0769e-01, -1.7150e-01,  2.0107e-01, -4.6051e-01,
         -2.5597e-01,  1.2257e-01, -2.0307e-01, -2.6036e-02, -1.0229e-01,
          1.7975e-01,  3.8284e-01,  1.1300e-01, -1.3566e-02, -4.8306e-01,
          2.9649e-01, -8.9245e-02, -2.2308e-01,  5.6348e-01, -2.1034e-01,
          3.7344e-01,  4.8375e-01,  3.3607e-01, -4.1791e-03, -8.3550e-03,
          1.5638e-01, -4.1402e-02, -6.5913e-02, -1.0038e-02, -1.1210e-01,
          1.7810e-01, -1.0573e-01, -8.

# Switch to deploy

In [7]:
for module in model.modules():
    if hasattr(module, 'switch_to_deploy'):
        module.switch_to_deploy()

In [8]:
from torchinfo import summary
summary(model, input_size=(1, 3, 112, 112), col_names=('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'))

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
RepVGG                                   [1, 3, 112, 112]          [1, 512]                  --                        --                        --
├─RepVGGBlock: 1-1                       [1, 3, 112, 112]          [1, 48, 56, 56]           --                        --                        --
│    └─Conv2d: 2-1                       [1, 3, 112, 112]          [1, 48, 56, 56]           1,344                     [3, 3]                    4,214,784
│    └─Identity: 2-2                     [1, 48, 56, 56]           [1, 48, 56, 56]           --                        --                        --
│    └─ReLU: 2-3                         [1, 48, 56, 56]           [1, 48, 56, 56]           --                        --                        --
├─ModuleList: 1-2                        --                        --                        --   

In [9]:
model = model.to(device)
with torch.no_grad():
    feat_new = model(img)
feat_new

tensor([[ 3.2363e-01,  4.7822e-02,  3.1942e-02,  3.1439e-01,  2.9077e-01,
         -1.6843e-01,  2.1655e-02, -1.9590e-01,  2.0792e-02,  3.2545e-01,
          4.0848e-01,  1.9912e-01,  1.6813e-01,  2.2000e-01,  1.5470e-01,
          4.3031e-01,  4.2403e-02, -1.8386e-01,  4.2585e-01,  4.6259e-01,
         -3.7674e-01,  2.8831e-03, -8.3697e-02,  8.6850e-02, -5.1917e-02,
          1.5622e-02,  1.2021e-01,  2.3824e-01, -2.7435e-01, -3.3648e-01,
         -6.2325e-02,  9.0248e-02, -1.9050e-01,  1.3745e-01, -1.2862e-02,
          5.9203e-01, -3.0769e-01, -1.7150e-01,  2.0106e-01, -4.6051e-01,
         -2.5597e-01,  1.2257e-01, -2.0307e-01, -2.6036e-02, -1.0229e-01,
          1.7975e-01,  3.8284e-01,  1.1300e-01, -1.3566e-02, -4.8306e-01,
          2.9649e-01, -8.9244e-02, -2.2308e-01,  5.6348e-01, -2.1034e-01,
          3.7344e-01,  4.8375e-01,  3.3608e-01, -4.1791e-03, -8.3555e-03,
          1.5638e-01, -4.1402e-02, -6.5912e-02, -1.0038e-02, -1.1210e-01,
          1.7810e-01, -1.0573e-01, -8.

In [10]:
diff = torch.abs(feat - feat_new)

In [12]:
torch.min(diff)

tensor(0.)

In [13]:
torch.max(diff)

tensor(5.0664e-07)